# SQL tutorial using a database of drinks and bars

## Things we will be doing:
- Creating a PostgreSQL database using the `psql` shell tool
- Importing existing data to fill the database (the `drinks.sql` file in this repository)
- Using the `psycopg2` Python package to interact with the database (we could have alternatively used a dedicated SQL IDE like SQL Workbench)
- Quering the database to explore SQL concepts such as:
    - Filtering, ordering, limiting, etc.
    - Joining tables
    - Grouping records
    - Aggregate functions
    
## Requirements:
- Install PostgreSQL (and potentially give the `postgres` user a password)
- Install the `psycopg2` Python package in your anaconda virtual environment of choice. **Note**: you will need to install other dependencies beforehand. Look up which ones for your operating system.
    - Using conda: `conda install -c anaconda psycopg2`
    - Using pip: `pip install psycopg2`

# Creating the database using `psql`

1. Start up your terminal.
2. Switch to the "postgres" user that was created automatically when installing PostgreSQL (find the equivalent command for Windows). \
`sudo -u postgres -i`
3. Start the `psql` shell tool. \
`psql`
4. List the PostgreSQL databases that exist on your computer (press "q" when you want to exit the list view). \
`\l`
5. Create a new empty database called "drinks". \
`CREATE DATABASE drinks;`
4. Confirm that the "drinks" database was created (press "q" when you want to exit the list view). \
`\l`
5. We can now leave the `psql` tool. \
`exit`
5. And also log out of the "postgres" user. \
`exit`

# Create a Python connection to the database and load in some data from the `drinks.sql` file

In [4]:
import psycopg2
import pandas as pd

# Create a connection to the drinks database we just made (substitute the password you set for the "postgres" user)
con = psycopg2.connect(database='drinks', user='lhl_student', password='lhl_password',
                       host='127.0.0.1', port='5432')
cur = con.cursor()

cur.execute(open('../../migrations/drinks.sql', 'r').read())
con.commit()

In [5]:
# Using triple quotations will ensure that everything inside the string
# is read as a character (i.e. no need to use escape characters)
query = """
SELECT * 
FROM drinks
LIMIT 5;
"""

# You first execute the query, then get it's result.
# Note: if you try to chain .execute().fetchall() in the same statement,
# the database my not have time to execute the query in time, which will
# result in you getting an error.
cur.execute(query)
response = cur.fetchall()

# Let's look at the format of the response we get back
print(f'This is the raw response we get back:\n{response}\n')

# You can alternatively use pandas to get a nicely formatted DataFrame
pandas_response = pd.read_sql_query(query, con)
print('This is the nicely structured pandas response:\n')
pandas_response
con.close()

This is the raw response we get back:
[('drink 1', 'cocktail'), ('drink 2', 'wine'), ('drink 3', 'rum'), ('drink 4', 'cocktail'), ('drink 5', 'cocktail')]

This is the nicely structured pandas response:



In [9]:
# Before doing anything else, let's create a function out
# of the things we're doing above
def execute_query(query_string, return_pandas=True, limit=' LIMIT 20;'):
    con = psycopg2.connect(database='drinks', user='lhl_student', password='lhl_password',
                       host='127.0.0.1', port='5432')
    cur = con.cursor()
    if limit:
        query_string+=limit
    if return_pandas:
        response = pd.read_sql_query(query_string, con)
    else:
        cur.execute(query_string)
        response = cur.fetchall()
    con.close()
    return response


# Let's try out our function to make sure it does
# the same as what we have above

query = """
SELECT * 
FROM drinks
"""
execute_query(query, limit=None)

,drink_id,type
0,drink 1,cocktail
1,drink 2,wine
2,drink 3,rum
3,drink 4,cocktail
4,drink 5,cocktail
5,drink 6,wine
6,drink 7,cocktail
7,drink 8,whisky
8,drink 9,rum
9,drink 10,wine


# Some SQL references before we get started
![](../../assets/sql-cheat-sheet.png)

## Order of operations
![](../../assets/order_of_operations.png)

# Time to start writing some queries

## Problem 1
Get the bar name and average price of drinks at each bar.

In [20]:
execute_query("""
SELECT bar, AVG(price) 
FROM menu_items
GROUP BY bar
""")

,bar,avg
0,bar 1,83.678519
1,bar 10,60.572635
2,bar 11,31.562558
3,bar 18,105.214779
4,bar 4,49.250977
5,bar 9,52.381706
6,bar 14,93.438857
7,bar 5,100.603858
8,bar 17,107.329703
9,bar 3,129.809513


## Problem 2
Get the bars with the top 5 average prices.

In [22]:
execute_query("""
SELECT bar, AVG(price) 
FROM menu_items
GROUP BY 1
ORDER BY 2 DESC
""", limit='LIMIT 5')

,bar,avg
0,bar 12,152.071032
1,bar 13,150.560439
2,bar 3,129.809513
3,bar 16,121.364365
4,bar 17,107.329703


## Problem 3
Get the bar with the cheapest drink, along with the drink type and price.

In [26]:
execute_query("""
SELECT bar, drinks.type, price 
FROM menu_items
LEFT JOIN drinks ON menu_items.drink_id = drinks.drink_id
ORDER BY price ASC
""", limit='LIMIT 1;')

,bar,type,price
0,bar 18,soda,3.47789


## Problem 4
Get the number of beers sold by each bar in descending order (number of beers sold, not the number of beers on their menu).

In [31]:
execute_query("""
SELECT bar, COUNT(*) as beersSold
FROM orders
LEFT JOIN drinks ON orders.drink_id = drinks.drink_id
WHERE drinks.type = 'beer'
group by 1
order by 2 desc
""")

,bar,beerssold
0,bar 20,62
1,bar 5,31
2,bar 10,27
3,bar 11,27
4,bar 3,26
5,bar 2,24
6,bar 17,19


## Problem 5
For each person, find the bar they visit, and the type(s) and price(s) of the drink(s) they drink during those visits.

In [39]:
execute_query("""
select 
distinct 
o.person, o.bar, d.type, mi.price
FROM orders o
LEFT JOIN menu_items mi on o.drink_id = mi.drink_id and o.bar = mi.bar
LEFT JOIN drinks d on d.drink_id = o.drink_id
""")

,person,bar,type,price
0,person 17,bar 4,whisky,65.86470
1,person 4,bar 8,whisky,45.60510
2,person 26,bar 1,wine,313.76800
3,person 34,bar 6,cocktail,12.44800
4,person 94,bar 9,whisky,74.95700
5,person 69,bar 17,whisky,23.55550
6,person 50,bar 3,whisky,52.18730
7,person 6,bar 18,soda,3.47789
8,person 57,bar 3,soda,7.08771
9,person 15,bar 1,wine,313.76800


## Bonus: Problem 6
Your restraunt chain is loosing business, how can we find out what the last bar a person visited, can we attribute this bar to being a root cause to why our customers are leaving ?

In [46]:
execute_query("""
select * from (
select person, date, bar, row_number() over (partition by person order by date desc) as vist_number
from orders
) as t
where t.vist_number = 1
""", limit=None)

,person,date,bar,vist_number
0,person 1,2016-10-22,bar 19,1
1,person 10,2016-10-31,bar 19,1
2,person 100,2016-10-17,bar 12,1
3,person 11,2016-10-29,bar 7,1
4,person 12,2016-10-28,bar 6,1
...,...,...,...,...
94,person 95,2016-10-14,bar 13,1
95,person 96,2016-10-31,bar 1,1
96,person 97,2016-10-05,bar 1,1
97,person 98,2016-10-18,bar 11,1


In [60]:
execute_query("""
select max(date), min(date)
from orders
""")

,max,min
0,2016-10-31,2016-10-01


## Drinks Database Schema Diagram
<img src="../../assets/drinks_schema_diagram.png" width="750"/>

# Below are my answers

## Answer 1

In [4]:
execute_query("""
SELECT bar, AVG(price) AS avg_price
FROM menu_items
GROUP BY bar
""")

,bar,avg_price
0,bar 1,83.678519
1,bar 10,60.572635
2,bar 11,31.562558
3,bar 12,152.071032
4,bar 13,150.560439
5,bar 14,93.438857
6,bar 15,90.993793
7,bar 16,121.364365
8,bar 17,107.329703
9,bar 18,105.214779


## Answer 2

In [5]:
execute_query("""
SELECT bar, AVG(price) AS avg_price
FROM menu_items
GROUP BY bar
ORDER BY avg_price DESC
""", limit = 'LIMIT 5')

,bar,avg_price
0,bar 12,152.071032
1,bar 13,150.560439
2,bar 3,129.809513
3,bar 16,121.364365
4,bar 17,107.329703


## Answer 3

In [6]:
execute_query("""
SELECT bar, drink_id, price
FROM menu_items
ORDER BY price ASC
""", limit='LIMIT 1;')

,bar,drink_id,price
0,bar 18,drink 43,3.47789


## Answer 4

In [7]:
execute_query("""
SELECT orders.bar, SUM(orders.quantity) as beers_sold 
FROM orders
JOIN drinks ON drinks.drink_id = orders.drink_id
WHERE drinks.type LIKE '%beer%'
GROUP BY orders.bar
ORDER BY beers_sold DESC
""")

,bar,beers_sold
0,bar 20,176
1,bar 5,111
2,bar 3,108
3,bar 11,80
4,bar 2,79
5,bar 10,66
6,bar 17,36


## Answer 5

In [8]:
execute_query("""
SELECT o.person, o.bar, d.type, d.drink_id, h.price
FROM orders AS o
JOIN menu_items AS h ON (o.drink_id = h.drink_id AND o.bar = h.bar) 
JOIN drinks AS d ON o.drink_id = d.drink_id
GROUP BY o.person, o.bar, d.type, d.drink_id, h.price
""")

,person,bar,type,drink_id,price
0,person 81,bar 2,rum,drink 39,6.44976
1,person 33,bar 20,beer,drink 38,7.96127
2,person 86,bar 12,whisky,drink 34,50.32640
3,person 29,bar 6,rum,drink 9,15.32780
4,person 3,bar 7,soda,drink 14,7.46722
5,person 13,bar 2,beer,drink 33,7.54580
6,person 16,bar 8,whisky,drink 40,45.60510
7,person 36,bar 9,cocktail,drink 4,26.48720
8,person 77,bar 15,whisky,drink 13,53.23560
9,person 10,bar 7,whisky,drink 28,91.36450


# Remember to close your database connection when you're done.
**Note**: If you run this cell then try to run queries, it won't work.
You will have to re-run the cell that initialized the connection.

In [9]:
con.close()